In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from getpass import getpass
from fuzzywuzzy import fuzz
import time
import pandas as pd
import chardet


####################### Build Grade Posting Functions ###########################

def entering_grades(fuzzy=True, gradebook=True, grade_col_name=True):
    # Count Students and iterate over them 
    n_students = len(driver.find_elements_by_xpath('//*[@id="pg0_V__stuActive__gvResults"]/tbody/tr'))
    for rows in range(2, n_students+1):
        pre = '//*[@id="pg0_V__stuActive__gvResults"]/tbody/tr['
        post_name = ']/td[2]'
        post_grade = ']/td[4]/input'
        xpath_name = pre + str(rows) + post_name
        xpath_grade = pre + str(rows) + post_grade
        #Fetch Name 
        name = driver.find_element_by_xpath(xpath_name).text
        # Match name to data set
        match = match_name(name, class_roster.gb_name, fuzzy)[0]
        #print(name)
        #print(match)
        #Fetch Grade for Name
        #grade = Matched_2[Matched_2['gb_name']==name].Score_1*100 # v.0
        grade = gradebook[gradebook['gb_name']==match][grade_col_name]*100 # v.1
        #try: 
            #grade = gradebook[gradebook['gb_name']==name][grade_col_name]*100 # New Code v.2
        try:
            grade = grade.round(0).values[0]
            #print(name + ' - ' + match + ' - ' + str(grade) )
            #Enter Grade for Name
            grade_textbox = driver.find_element_by_xpath(xpath_grade)
            driver.find_element_by_xpath(xpath_grade).clear()
            grade_textbox.send_keys(str(grade))
        except:
            print('not found')

        
    # Submit Grades
    submit_button = driver.find_element_by_xpath('//*[@id="pg0_V__gbtnSubmit"]')
    submit_button.click()
    # Return to page = Academics
    Acad_button = driver.find_element_by_xpath('//*[@id="main-nav"]/li[3]/a')
    Acad_button.click()
        
    return



def delete_grades(fuzzy=True, gradebook=True, grade_col_name=True):
    # Count Students and iterate over them 
    n_students = len(driver.find_elements_by_xpath('//*[@id="pg0_V__stuActive__gvResults"]/tbody/tr'))
    for rows in range(2, n_students+1):
        pre = '//*[@id="pg0_V__stuActive__gvResults"]/tbody/tr['
        post_name = ']/td[2]'
        post_grade = ']/td[4]/input'
        xpath_name = pre + str(rows) + post_name
        xpath_grade = pre + str(rows) + post_grade
        #Fetch Name 
        name = driver.find_element_by_xpath(xpath_name).text
        # Match name to data set
        match = match_name(name, class_roster.roster_name, fuzzy)[0]

        #Fetch Grade for Name
        #grade = Matched_2[Matched_2['gb_name']==name].Score_1*100
        grade = ''
        
        print(name + ' - ' + match + ' - ' + str(grade) )
        #Enter Grade for Name
        grade_textbox = driver.find_element_by_xpath(xpath_grade)
        driver.find_element_by_xpath(xpath_grade).clear()
        grade_textbox.send_keys(str(grade))
        
    # Submit Grades
    submit_button = driver.find_element_by_xpath('//*[@id="pg0_V__gbtnSubmit"]')
    submit_button.click()
    # Return to page = Academics
    Acad_button = driver.find_element_by_xpath('//*[@id="main-nav"]/li[3]/a')
    Acad_button.click()
        
    return



def link_by_name(TargetName=True, table_path=True, ):
    '''
    METHOD RETIRED in favor of find_element_by_link_text()

    Assignments are on an online Table, listed by name, hyperlinked.  This Utility gets that link by assignment name
    so that links can be dynamically retrieved based on assignment name
    '''
    n_assignments=len(driver.find_elements_by_xpath(table_path))
    print('link_by_name ' + TargetName)
    for row in range(2,n_assignments+1):
        front_assn = '//*[@id="pg0_V__assignmentView__rptAssignments_ctl00__assignGrid__gvAssignments"]/tbody/tr['
        end_assn = ']/td[1]/a'
        path_assn = front_assn + str(row) + end_assn
        assn_name = driver.find_element_by_xpath(path_assn).text
        if assn_name==TargetName:
            print('found assignment'+str(TargetName) + ' from list')
            return path_assn
        else:
            print('still scrapping down the list of assignment names on table')



def enter_grades_navigate(course=True, assignment=True):
    '''
    course must be in the same format as on Academics Page; thus I scrape this off the Academics page and enter it in here
    automatically for my course.  Other teachers may want to have individual control, which needs to be built-in. 
    '''
    print('enter_grades_navigate '+ assignment)
    # Navigate to grade entry
    coursework_button = driver.find_element_by_link_text('Coursework').click()
    time.sleep(1)
    # Dynamic Button to Assignment
    table_path = '//*[@id="pg0_V__assignmentView__rptAssignments_ctl00__assignGrid__gvAssignments"]/tbody/tr'


    time.sleep(1)
    assn_path = driver.find_element_by_link_text(assignment).click()
    '''
    if course=='ECO 220 (O)':
        assn_path = driver.find_element_by_link_text(assignment).click()
    else:
        assn_path = link_by_name(TargetName=assignment, table_path=table_path)   
        assn_button = driver.find_element_by_xpath(assn_path)
        assn_button.click() 
    #link_by_name METHOD ALMOST RETIRED in favor of code below but somehow the code below geeks out
    '''
    

    time.sleep(1)
    Edit_link = driver.find_element_by_xpath('//*[@id="pg0_V__subHeader__liFirstLink"]')
    Edit_link.click()
    return 

def Class_Roster(gradebook=True, course=True):
    # Get DataSet for Class
    class_roster = gradebook[gradebook['course']==course ] # extract only students from current class for fuzzy matching
    #print(class_roster)
    time.sleep(2)
    return class_roster

def Key_in_Grades(fuzzy=True, gradebook=True, grade_col_name=True):
    entering_grades(fuzzy=fuzzy, gradebook=gradebook, grade_col_name=grade_col_name)
    return

# Convert grades to numerics A = 2 ; B+ = 3 ; B = 4 ; C+ = 5 ; C = 6 ; D+ = 7 ; D = 8 ; F = 9
def numeric_grades(Midterm_grade):
    if (Midterm_grade>=0.9):
        return 2
    elif ((Midterm_grade>=0.85) and (Midterm_grade<0.9)):
        return 3
    elif (Midterm_grade>=0.80) and (Midterm_grade<0.85):
        return 4
    elif (Midterm_grade>=0.75) and (Midterm_grade<0.80):
        return 5
    elif (Midterm_grade>=0.70) and (Midterm_grade<0.75):
        return 6
    elif (Midterm_grade>=0.65) & (Midterm_grade<0.70):
        return 7
    elif (Midterm_grade>=0.60) & (Midterm_grade<0.65):
        return 8
    elif (Midterm_grade<0.6):
        return 9


def roman_grades(Midterm_grade):
    if (Midterm_grade>=0.9):
        return 'A'
    elif ((Midterm_grade>=0.85) and (Midterm_grade<0.9)):
        return 'B+'
    elif (Midterm_grade>=0.80) and (Midterm_grade<0.85):
        return 'B'
    elif (Midterm_grade>=0.75) and (Midterm_grade<0.80):
        return 'C+'
    elif (Midterm_grade>=0.70) and (Midterm_grade<0.75):
        return 'C'
    elif (Midterm_grade>=0.65) & (Midterm_grade<0.70):
        return 'D+'
    elif (Midterm_grade>=0.60) & (Midterm_grade<0.65):
        return 'D'
    elif (Midterm_grade<0.6):
        return 'F'

##################### MIDTERM EVALS
def Midterm_Evals():
    Matched['course'] = Matched.course.str.replace('(','')
    Matched['course'] = Matched.course.str.replace(')','')
    course = 'ECO 220 O'
    class_roster = Class_Roster(gradebook=Matched, course=course)

    fuzzy=30
    n_students = len(driver.find_elements_by_xpath('//*[@id="pg0_V_dgClassList"]/tbody/tr')) 
    for rows in range(1, n_students+1):
        pre = '//*[@id="pg0_V_dgClassList"]/tbody/tr['
        post_name = ']/td[2]'
        post_id = ']/td[3]'
        post_grade = ']/td[5]/select/option['
        end_grade = ']'
        xpath_name = pre + str(rows) + post_name
        xpath_id = pre + str(rows) + post_id

        #Fetch Name 
        name = driver.find_element_by_xpath(xpath_name).text
        #Fetch ID
        id = driver.find_element_by_xpath(xpath_id).text
        # Match name on website "name" to name in class roster and return matching name on roster = "match"
        
        #match = match_name(name, class_roster.gb_name, fuzzy)[0]
        #print('Name =' + name, 'Match =' + match)
        #Fetch Grade for Name
        #grade = Matched[Matched['gb_name']==match]['numeric_grade'].values[0]

        print(xpath_id)
        try:
            grade = Matched[Matched['ID']==id]['numeric_grade'].values[0]
            xpath_grade = pre  + str(rows) + post_grade + str(grade) + end_grade
        except:
            print('missing id'+ str(id))
        #print(xpath_grade)
        #print(xpath_grade)
        #Enter Grade for Name
        try:
            grade_enter = driver.find_element_by_xpath(xpath_grade)
            grade_enter.click()
            #submit_button = driver.find_element_by_xpath('//*[@id="pg0_V_btnSave"]')
        except:
            print('skipping one')


def Final_Grade():
    Matched['course'] = Matched.course.str.replace('(','')
    Matched['course'] = Matched.course.str.replace(')','')
    course = 'ECO 210 A'
    class_roster = Class_Roster(gradebook=Matched, course=course)

    fuzzy=30
    n_students = len(driver.find_elements_by_xpath('//*[@id="pg0_V_dgClassList"]/tbody/tr')) 
    for rows in range(1, n_students+1):
        pre = '//*[@id="pg0_V_dgClassList"]/tbody/tr['
        post_name = ']/td[2]'
        post_grade = ']/td[5]/select/option['
        end_grade = ']'
        xpath_name = pre + str(rows) + post_name


        #Fetch Name 
        name = driver.find_element_by_xpath(xpath_name).text
        # Match name on website "name" to name in class roster and return matching name on roster = "match"
        match = match_name(name, class_roster.gb_name, fuzzy)[0]

        #Fetch Grade for Name
        grade = Matched[Matched['gb_name']==match]['roman_grade'].values[0]

        xpath_grade = pre  + str(rows) + post_grade + str(grade) + end_grade
        print(xpath_grade)
        #print(xpath_grade)
        #Enter Grade for Name
        try:
            grade_enter = driver.find_element_by_xpath(xpath_grade)
            grade_enter.click()
            #submit_button = driver.find_element_by_xpath('//*[@id="pg0_V_btnSave"]')
        except:
            print('skipping one')



#Scraping Functions

In [ ]:
def initiate_scrape():
    username = input('Enter Username:')
    password = getpass('Enter Password')

    driver = webdriver.Chrome(webdriver_path)
    driver.get('https://my.newberry.edu/ICS/')

    username_textbox = driver.find_element_by_id('userName')
    username_textbox.send_keys(username)

    password_textbox = driver.find_element_by_xpath('//*[@id="password"]')
    password_textbox.send_keys(password)

    login_button = driver.find_element_by_xpath('//*[@id="siteNavBar_btnLogin"]')
    login_button.click()
    Academic_button = driver.find_element_by_xpath('//*[@id="main-nav"]/li[3]/a')
    Academic_button.click()
    return driver


def scrape_roster(course=True):

    #This Works from inside the course Tab Page ONLY
    Roster_button = driver.find_element_by_xpath('//*[@id="this-context-pages"]/li[2]/a')
    Roster_button.click()

    Print_Roster = driver.find_element_by_xpath('//*[@id="pg0_V_ViewPrintableRosterLink"]')
    Print_Roster.click()

    print('Scrape Roster Names and GradeBook Names') # --> "roster_gb"
    dict_roster=[]
    count = len(driver.find_elements_by_xpath('//*[@id="pg0_V_ItemsGrid"]/tbody/tr'))
    for rows in range(1, count+1):
        part_1 = '//*[@id="pg0_V_ItemsGrid"]/tbody/tr['
        part_2_name = ']/td[3]'
        part_2_id = ']/td[2]'
        final_name = part_1 + str(rows) + part_2_name
        final_id = part_1 + str(rows) + part_2_id
        ID = driver.find_element_by_xpath(final_id).text  #Student ID - Clean
        name = driver.find_element_by_xpath(final_name).text  #Name - Uncleaned
        dict_={}
        dict_.update({'name': name, 'ID': ID})
        dict_roster.append(dict_)
    roster_gb = pd.DataFrame(dict_roster)
    roster_names=roster_gb.name.str.split('\\n', expand=True)
    roster_names= roster_names.rename(columns={0:'name_gb', 1:'junk2'})
    roster_names[['junk','roster_name']]= roster_names.junk2.str.split('for ', expand=True)
    roster_names.drop(columns={'junk', 'junk2'})
    roster_gb.drop(columns={'name'})
    roster_gb[['roster_name', 'gb_name']]=roster_names[['roster_name','name_gb']]


    #print('Scrape Emails and Roster Names') # --> "emails"

    selectALL_button = driver.find_element_by_xpath('//*[@id="pg0_V_ItemsGrid_sec1_row0_SelectAll"]')
    selectALL_button.click()

    Email_button = driver.find_element_by_xpath('//*[@id="pg0_V_EmailAll"]')
    Email_button.click()
    emails=[driver.find_element_by_xpath('//*[@id="pg0_V_spanRecipients"]').text]
    emails=pd.DataFrame(emails)
    emails = (emails.assign(junk = emails[0].str.split(','))
             .explode('junk')
             .reset_index(drop=True))
    emails = emails.drop(columns=[0])
    emails[['roster_name','email']]= emails.junk.str.split(' \(', expand=True)
    emails['email'] = emails['email'].str.replace(')', '')
    emails['email'] = emails['email'].str.replace(' ', '')
    emails = emails[['roster_name', 'email']]

    #print('Clean up and merge roster_gb and emails') #  --> full_WolfDen
    roster_gb['roster_name'] = roster_gb.roster_name.str.lstrip()
    roster_gb['roster_name'] = roster_gb.roster_name.str.rstrip()
    
    emails['roster_name'] = emails.roster_name.str.rstrip()
    emails['roster_name'] = emails.roster_name.str.lstrip()
    #print(roster_gb['roster_name'] + ' ' +  emails['roster_name'] )
    full_WolfDen = roster_gb.merge(emails, how='left', on='roster_name')
    full_WolfDen = full_WolfDen[['roster_name', 'email', 'ID', 'gb_name']]
    full_WolfDen['ID'] = full_WolfDen['ID'].str.replace(' ', '')
    full_WolfDen
    
    #print('Return to Academics Page to scrape the next class on the list')
    driver.find_element_by_link_text('Academics').click()
    
    
    return full_WolfDen


def scrape_all():
    driver.find_element_by_link_text('Academics').click()
    length = len(driver.find_elements_by_xpath('//*[@id="tblCoursesSched"]/tbody/tr')) # number of rows in table 
    course_list = []
    full_WolfDen = pd.DataFrame()
    for rows in range(2,length+1):
        front = '//*[@id="tblCoursesSched"]/tbody/tr['
        end = ']/td[1]/a[1]'
        path = front + str(rows) + end
        course = driver.find_element_by_xpath(path).text
        course_dict_ = {}
        course_dict_.update({'course': course})
        course_list.append(course_dict_)

        # Now click on course button in table
        button = driver.find_element_by_xpath(path)
        button.click()

        if rows==2:
            print('now on ' + course)
            full_WolfDen = scrape_roster()
            full_WolfDen['course']=course
        else:
            print('now on ' + course)
            full_WolfDen2 = scrape_roster()
            full_WolfDen2['course']=course 
            full_WolfDen = pd.concat([full_WolfDen, full_WolfDen2])

    # clean up email - matching variable with Pearson      
    #full_WolfDen['email'] = full_WolfDen.email.str.lower()
    #full_WolfDen['email'] = full_WolfDen.email.str.replace(' ', '')  
    return full_WolfDen

def open_csv(path_file=True):
    # Strange Encoding when opening CSV file so this is the work around: 
    with open(path_file, 'rb') as f:
        result = chardet.detect(f.read()) # or readline if the file is large
        df_name=pd.read_csv(path_file,
                       encoding=result['encoding'])
    return df_name


def clean_pearson(df=True): 
    #CLEAN UP Pearsons
    # df_Pearson = Pearson Grades
    df_Pearson = df.rename(columns={'Last name': 'lastname', 'First name': 'firstname', 'Email': 'email'})
    df_Pearson['lastname'] = df_Pearson['lastname'].str.lower()
    df_Pearson['firstname'] = df_Pearson['firstname'].str.lower()
    df_Pearson['Course']=df_Pearson.Course.astype(str)
    # Names in WolfDen are more complete than in Pearson. Thus create full name in Pearson and match with "name" in WolfDen
    df_Pearson['name'] = df_Pearson['firstname']+ ' ' + df_Pearson['lastname']
    return df_Pearson

def Pearson_cnames(assignment_list=True):
    assignments = assignment_list
    column_names = ['email', 'lastname', 'firstname', 'name','Log-in','Course']
    for i in range(0, len(assignments)):
        column_names.append(assignments[i])
    return column_names


#def email_match(WolfDen=True, Pearson=True, assignment_names=True):  # Need to dynamically expand assignment list to make it generalizable 
def email_match(WolfDen=True, Pearson=True): # or just remove assignment_names
    df_WolfPearson = WolfDen.merge(Pearson,
                                     left_on='email',
                                     right_on='email',
                                     how='outer',
                                     suffixes=['','_Pearson'])
    #suffixes adds "_Pearson to variables that have the same name as WolfDen"

    # nm_Wolf = No Match for WolfDen Records 
    nm_Wolf = df_WolfPearson[df_WolfPearson.name.isnull()]
    nm_Wolf = nm_Wolf[['roster_name', 'email','ID','gb_name','course', 'course_n']]
        
    # nm_Pearson = No Match for Pearson Records
    nm_Pearson = df_WolfPearson[df_WolfPearson.roster_name.isnull()]
    #column_names = Pearson_cnames(assignment_list=assignment_names)
    #nm_Pearson = nm_Pearson[column_names]
    nm_Pearson=nm_Pearson.drop(columns='roster_name')
    nm_Pearson=nm_Pearson.dropna(axis=1,how='all')

    #Save MATCHES
    Matched_Pearson = df_WolfPearson[pd.notnull(df_WolfPearson.name)]
    Matched_Pearson = Matched_Pearson[pd.notnull(Matched_Pearson.roster_name)]
    
    return nm_Wolf, nm_Pearson, Matched_Pearson


def match_name(name, name_list, min_score=0):
    max_score = -1 #in event no match
    max_name = "" # returns empty name in case of no match
    for name2 in name_list:
        score = fuzz.ratio(name, name2)
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score           
    return(max_name, max_score)


def matched_namelist(nl_Pearson=True, nl_WolfDen=True, fuzzy=True):
    # 0 < fuzzy parameter < 100
    dict_list = []
    for name in nl_Pearson['name']:
        match = match_name(name, nl_WolfDen.roster_name, fuzzy)
        dict_ = {}
        dict_.update({'name' : name})
        dict_.update({'roster_name' : match[0]})
        dict_.update({'score' : match[1]})
        dict_list.append(dict_)
    namelist = pd.DataFrame(dict_list)
    return namelist


def fuzzyname(nm_Pearson=True, nm_WolfDen=True, fuzz_cut=True):
    #Fuzzy Match on Name
    
    namelist= matched_namelist(nl_Pearson=nm_Pearson, nl_WolfDen=nm_WolfDen, fuzzy=fuzz_cut) # match on namelist for unmatched

    print('VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACy')
    print(namelist)
    #print('VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACY VISUALLY CHECK NAMELIST FOR ACCURACy')

    # Now nm_Pearson has the key "roster_name" - makes these people in Pearson now linkable to WolfDen
    df = namelist.merge(nm_Pearson,
                        left_on='name',
                        right_on='name',
                        how='outer')
    # Link to WolfDen
    df2 = df.merge(nm_WolfDen,
                  left_on='roster_name',
                  right_on='roster_name',
                  how='outer')
    df2 = df2.rename(columns={'email_x':'email_alt', 'email_y':'email'})
    return df2 

def enter_single(df, name, assignment, score):
    df.loc[df['roster_name'].str.contains(name), assignment]=score
    return

def WolfDen_session():
    
    username = input('Enter Username:')
    password = 'b&Km465$hZz3y'
    #password = getpass('Enter Password')
    driver = webdriver.Chrome(webdriver_path)
    driver.get('https://my.newberry.edu/ICS/')

    username_textbox = driver.find_element_by_id('userName')
    username_textbox.send_keys(username)

    password_textbox = driver.find_element_by_xpath('//*[@id="password"]')
    password_textbox.send_keys(password)

    login_button = driver.find_element_by_xpath('//*[@id="siteNavBar_btnLogin"]')
    login_button.click()
    Academic_button = driver.find_element_by_xpath('//*[@id="main-nav"]/li[3]/a')
    Academic_button.click()
    return